In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE203409"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE203409"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE203409.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE203409.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE203409.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Dict, Any, Callable, Optional

# 1. Determine gene expression data availability
# From the background information, this appears to be a gene expression study
is_gene_available = True

# 2. Determine variable availability and create conversion functions
# Looking at the sample characteristics dictionary:
# - This is an in vitro cell line study (HaCaT cells)
# - There are different knockdowns (shC and shFLG) and treatments
# - No human age or gender data is present as this is a cell line study

# For trait, we can use the knockdown status (shC vs shFLG)
# shFLG represents filaggrin-insufficient cells which is relevant to allergies
trait_row = 1  # knockdown data is in row 1

def convert_trait(value: str) -> int:
    """Convert knockdown status to binary trait."""
    if value is None:
        return None
    # Extract value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: shFLG (filaggrin-insufficient) = 1, shC (control) = 0
    if "shFLG" in value:
        return 1  # Filaggrin-insufficient (associated with allergies)
    elif "shC" in value:
        return 0  # Control
    return None

# Age and gender are not applicable as this is a cell line study
age_row = None
gender_row = None
convert_age = None
convert_gender = None

# 3. Save metadata about dataset usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait is available
if trait_row is not None:
    # Create sample characteristics dictionary from the provided output
    sample_characteristics_dict = {
        0: ['cell line: HaCaT'], 
        1: ['knockdown: shC', 'knockdown: shFLG'], 
        2: ['treatment: Untreated', 'treatment: Histamine', 'treatment: Amphiregulin', 'treatment: IFNy', 'treatment: IL-4/IL-13', 'treatment: Cysteine', 'treatment: Derp1/cysteine', 'treatment: Derp2'], 
        3: ['treatment compound concentration: N/A', 'treatment compound concentration: 1 ug/ml', 'treatment compound concentration: 50 ng/ml', 'treatment compound concentration: 50 ng/ml / 50 ng/ml', 'treatment compound concentration: 10 uM', 'treatment compound concentration: 100 nM / 10 uM', 'treatment compound concentration: 100 nM']
    }
    
    # Create clinical_data from this dictionary - using proper transposition
    clinical_data = pd.DataFrame(sample_characteristics_dict).T
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of processed clinical data:")
    print(preview)
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the processed clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import os
import json
import pandas as pd
from typing import Callable, Optional, Dict, Any

# Assuming clinical_data is already available from previous steps
# Let's examine what we have in the clinical_data DataFrame
try:
    print("Clinical data preview:")
    print(clinical_data.head())
    print("\nClinical data shape:", clinical_data.shape)
    print("\nClinical data columns:", clinical_data.columns.tolist())
    
    # Print unique values for each row to analyze the content
    print("\nUnique values in clinical data:")
    for i in range(len(clinical_data)):
        unique_vals = clinical_data.iloc[i].unique()
        if len(unique_vals) < 10:  # Only print if there aren't too many unique values
            print(f"Row {i}: {unique_vals}")
        else:
            print(f"Row {i}: {len(unique_vals)} unique values")
except NameError:
    print("Clinical data not available from previous steps")
    clinical_data = pd.DataFrame()  # Create empty DataFrame if not available

# 1. Determine if gene expression data is available
# Look for indicators in the data structure and content
is_gene_available = True
# We'll assume gene expression data is available unless we find evidence to the contrary
# In a real scenario, we'd analyze clinical_data or other data to determine this

# 2. Variable availability and data type conversion
# Initialize as None, will be updated if found
trait_row = None
age_row = None
gender_row = None

# Examine clinical data to find rows containing trait, age, and gender information
if not clinical_data.empty:
    for i in range(len(clinical_data)):
        row_values = ' '.join(clinical_data.iloc[i].astype(str).tolist()).lower()
        
        # Look for allergy-related information
        if any(term in row_values for term in ['allergy', 'allergic', 'atopic', 'asthma', 'rhinitis']):
            trait_row = i
        
        # Look for age information
        if any(term in row_values for term in ['age', 'years old']):
            age_row = i
        
        # Look for gender/sex information
        if any(term in row_values for term in ['gender', 'sex', 'male', 'female']):
            gender_row = i

    # Check if the identified rows have varying values (not constant)
    if trait_row is not None:
        unique_values = clinical_data.iloc[trait_row].astype(str).unique()
        if len(unique_values) <= 1:
            trait_row = None  # Consider as not available if only one unique value

    if age_row is not None:
        unique_values = clinical_data.iloc[age_row].astype(str).unique()
        if len(unique_values) <= 1:
            age_row = None  # Consider as not available if only one unique value

    if gender_row is not None:
        unique_values = clinical_data.iloc[gender_row].astype(str).unique()
        if len(unique_values) <= 1:
            gender_row = None  # Consider as not available if only one unique value

# Define conversion functions
def convert_trait(value: str) -> Optional[int]:
    """Convert trait (allergy) value to binary format: 1 for present, 0 for absent."""
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Positive indicators
    if any(term in value for term in ['yes', 'positive', 'present', 'allergy', 'allergic', 'diagnosed', 'asthma', 'rhinitis', 'atopic']):
        return 1
    # Negative indicators
    elif any(term in value for term in ['no', 'negative', 'absent', 'control', 'healthy', 'normal']):
        return 0
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to continuous format."""
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Try to extract age as a number
    try:
        # Extract digits from the string
        import re
        numbers = re.findall(r'\d+\.?\d*', value)
        if numbers:
            return float(numbers[0])
        else:
            return None
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary format: 0 for female, 1 for male."""
    if pd.isna(value) or value is None:
        return None
    
    value = str(value).lower()
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if any(term in value for term in ['female', 'f', 'woman', 'girl']):
        return 0
    elif any(term in value for term in ['male', 'm', 'man', 'boy']):
        return 1
    else:
        return None

# 3. Save metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None and not clinical_data.empty:
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("\nSelected clinical features preview:")
    preview = preview_df(selected_clinical_df)
    print(preview)
    
    # Save to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("Clinical data not available or trait information not found. Skipping clinical feature extraction.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 5: Gene Identifier Review

In [ ]:
# Based on the gene identifiers shown, these are Illumina microarray probe IDs (ILMN_xxxxxxx format)
# They are not human gene symbols and will need to be mapped to gene symbols

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Identify the columns containing probe IDs and gene symbols in the gene annotation dataframe
# From the preview, we can see 'ID' contains probe identifiers matching the gene expression data
# The column 'Symbol' appears to contain gene symbols
probe_col = 'ID'
gene_col = 'Symbol'

# 2. Get the gene mapping dataframe by extracting the ID and Symbol columns
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Verify the mapping by printing a few rows
print("Gene mapping preview:")
print(gene_mapping.head())

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print the first few rows of the resulting gene expression data to verify the result
print("\nGene expression data after mapping:")
print(gene_data.head())
print("\nShape of gene expression data:", gene_data.shape)

# Save the processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
# First reload the gene data from the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Extract gene mapping from annotation
gene_annotation = get_gene_annotation(soft_file)
probe_col = 'ID'
gene_col = 'Symbol'
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

# Apply mapping to convert probe-level data to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Now normalize the gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Evaluate if we can proceed with linking clinical and genetic data
# From our analysis in previous steps, we know this is a cell line study with knockdown information
trait_row = 1  # knockdown status (shC vs shFLG)

# Define the trait conversion function since we need it
def convert_trait(value: str) -> int:
    """Convert knockdown status to binary trait."""
    if value is None:
        return None
    # Extract value after colon and strip whitespace
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: shFLG (filaggrin-insufficient) = 1, shC (control) = 0
    if "shFLG" in value:
        return 1  # Filaggrin-insufficient (associated with allergies)
    elif "shC" in value:
        return 0  # Control
    return None

is_trait_available = trait_row is not None

if is_trait_available:
    print("Extracting clinical features...")
    # Use the clinical_data obtained directly from the matrix file
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=None,  # Cell line study has no age
        convert_age=None,
        gender_row=None,  # Cell line study has no gender
        convert_gender=None
    )
    
    print("Clinical data preview:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to a CSV file
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data using the normalized gene data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape: {linked_data.shape}")
    
    # 3. Handle missing values in the linked data
    print("Handling missing values...")
    linked_data = handle_missing_values(linked_data, trait)
    print(f"Linked data shape after handling missing values: {linked_data.shape}")
    
    # 4. Check if trait is biased
    print("Checking for bias in trait distribution...")
    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    
else:
    print("No trait information available - this dataset cannot be used for trait-gene association analysis.")
    is_biased = True  # Set to True since we can't use this dataset without trait information
    linked_data = pd.DataFrame()  # Empty dataframe as placeholder

# 5. Final validation
note = "Dataset contains gene expression from HaCaT keratinocyte cell line with filaggrin knockdown (shFLG) vs control (shC). This represents an in vitro model relevant to allergies rather than direct human subject data."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to lack of trait information or other issues.")